In [1]:
import tensorflow as tf

# 프로그램 시작시에 가상 장치 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)  


2024-05-13 14:09:28.245679: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-13 14:09:28.883622: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


1 Physical GPUs, 1 Logical GPUs


2024-05-13 14:09:29.646189: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-13 14:09:29.674811: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-13 14:09:29.675071: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:
# import psutil
# import time

# def get_gpu_memory_usage():
#     try:
#         import pynvml
#         pynvml.nvmlInit()
#         handle = pynvml.nvmlDeviceGetHandleByIndex(0)
#         meminfo = pynvml.nvmlDeviceGetMemoryInfo(handle)
#         return meminfo.used / 1024**2  # 메모리 사용량을 MB 단위로 반환
#     except Exception as e:
#         print("Failed to get GPU memory usage:", e)
#         return None

# while True:
#     gpu_memory_usage = get_gpu_memory_usage()
#     if gpu_memory_usage is not None:
#         print("GPU Memory Usage: {:.2f} MB".format(gpu_memory_usage))
#     time.sleep(1)


In [3]:
# import tensorflow as tf

# # GPU 리밋 설정
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         # GPU에 리밋을 걸어둡니다.
#         for gpu in gpus:
#             tf.config.experimental.set_virtual_device_configuration(
#                 gpu,
#                 [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])  # 원하는 메모리 리밋으로 설정
#         logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#     except RuntimeError as e:
#         print(e)

# # CPU만 사용하도록 설정
# tf.config.set_visible_devices([], 'GPU')

# # 이후에 모델 생성과 훈련을 진행합니다.


In [4]:
# from numba import cuda

# device = cuda.get_current_device(); device.reset()

In [2]:
import tensorflow as tf

import numpy as np

import pandas as pd

import os

from tensorflow.keras.layers import GlobalAveragePooling2D,Dense, Dropout
from tensorflow.keras.models import Sequential

from tensorflow.keras.applications.vgg16 import VGG16

from tensorflow.keras.applications.vgg16 import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#     # Currently, memory growth needs to be the same across GPUs
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#         logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")

#     except RuntimeError as e:
#         print(e)        


1 Physical GPUs, 1 Logical GPUs


2024-05-13 14:05:48.667227: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-13 14:05:48.697080: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-13 14:05:48.697381: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
seed= 13
tf.random.set_seed(seed)
np.random.seed(seed)

In [4]:
train_df = pd.DataFrame({"file": os.listdir('/home/addinedu/dev_ws/data/machine/cat_dog/train/')})
train_df["label"] = train_df["file"].apply(lambda x: x.split(".")[0])
# train_df.head()

In [9]:
# len(train_df)

In [5]:
test_df = pd.DataFrame({'file': os.listdir('/home/addinedu/dev_ws/data/machine/cat_dog/test1')})

# test_df.head()

In [11]:
# test_df.size

In [6]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(train_df,
test_size = 0.2,
stratify = train_df["label"],
random_state = 13)

In [7]:
train_datagen = ImageDataGenerator(
rotation_range = 15,
horizontal_flip = True,
preprocessing_function = preprocess_input
)

val_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

In [8]:
FILES = '/home/addinedu/dev_ws/data/machine/cat_dog/'
batch_size = 16
train_generator = train_datagen.flow_from_dataframe(
dataframe = train_data,
directory = FILES + 'train/',
x_col = 'file',
y_col = "label",
class_mode = "categorical",
target_size = (224, 224),
batch_size = batch_size,
seed = 13,
)

Found 20000 validated image filenames belonging to 2 classes.


In [9]:
val_generator = val_datagen. flow_from_dataframe(
dataframe = val_data,
directory = FILES + "train/",
x_col = "file",
y_col = "label",
class_mode = "categorical",
target_size = (224, 224),

batch_size = batch_size,
seed = 13,

shuffle = False
)

Found 5000 validated image filenames belonging to 2 classes.


In [10]:
base_model = VGG16(
weights = "imagenet",
input_shape = (224, 224, 3),

include_top = False
)

In [17]:
# base_model.summary()

In [11]:
for layers in base_model.layers:
    layers. trainable = False

# base_model.summary ()

In [12]:
def vgg16_pretrained():
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(100, activation="relu"),
        Dropout(0.4),
        Dense(64, activation='relu'),
        Dense(2, activation='softmax')
    ])

    return model


In [13]:
tf.keras.backend.clear_session()

In [14]:
model = vgg16_pretrained()

model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

# model.summary()


In [15]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
monitor = "val_accuracy",
patience = 2,
verbose = 1,
factor = 0.5,
min_lr = 0.001
)

In [16]:
early_stopping = tf.keras.callbacks.EarlyStopping(
monitor = 'val_accuracy',
patience = 5,
verbose = 1,
mode = 'max',
)

In [17]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    monitor='val_accuracy',
    filepath='catdog_vgg16_{epoch:02d}-{val_accuracy:.6f}.weights.h5',
    verbose=1,
    save_best_only=True,
    save_weights_only=True
)

In [25]:
# import tensorflow as tf

# # GPU 리밋 설정
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         # GPU에 리밋을 걸어둡니다.
#         for gpu in gpus:
#             tf.config.experimental.set_virtual_device_configuration(
#                 gpu,
#                 [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])  # 원하는 메모리 리밋으로 설정
#         logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#     except RuntimeError as e:
#         print(e)

# # CPU만 사용하도록 설정
# tf.config.set_visible_devices([], 'GPU')

# # 이어서 모델 생성 및 훈련 코드를 작성하세요.


In [18]:
from numba import cuda

device = cuda.get_current_device(); device.reset()

In [19]:
history = model.fit(
train_generator,
epochs = 10,
validation_data = val_generator,
validation_steps = val_data.shape[0] // batch_size,
steps_per_epoch = train_data.shape[0] // batch_size,
callbacks = [reduce_lr, early_stopping, checkpoint]
)

: 

In [ ]:
import psutil
import time

def get_gpu_memory_usage():
    try:
        import pynvml
        pynvml.nvmlInit()
        handle = pynvml.nvmlDeviceGetHandleByIndex(0)
        meminfo = pynvml.nvmlDeviceGetMemoryInfo(handle)
        return meminfo.used / 1024**2  # 메모리 사용량을 MB 단위로 반환
    except Exception as e:
        print("Failed to get GPU memory usage:", e)
        return None

while True:
    gpu_memory_usage = get_gpu_memory_usage()
    if gpu_memory_usage is not None:
        print("GPU Memory Usage: {:.2f} MB".format(gpu_memory_usage))
    time.sleep(1)


In [2]:
import tensorflow as tf
tf.__version__
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2024-05-13 14:38:44.797296: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-13 14:38:45.465465: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-05-13 14:38:46.357343: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-13 14:38:46.385823: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. 

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16630718746180704922
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 750977024
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 1184374186561541952
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
 xla_global_id: 416903419]